In [1]:
!pip install transformers
!pip install sacrebleu sentencepiece
!pip install huggingface_hub
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=89

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import re
import glob
import pandas as pd
from natsort import natsorted

from huggingface_hub import notebook_login
from transformers import pipeline

In [4]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Traduzione

Configuro modelli HuggingFace per la traduzione da Inglese a Italiano

In [5]:
# Modello Traduzione Inglese-Italiano
en_model_checkpoint = "Helsinki-NLP/opus-mt-it-de"
translator_en = pipeline("translation", model=en_model_checkpoint)

### Configuro Classe per la traduzione

In [6]:
class Translate:
  def __init__(self,
               orig_filenamepath, 
               trad_filenamepath, 
               translator_en):
    
    self.orig_filenamepath = orig_filenamepath
    self.trad_filenamepath = trad_filenamepath

    self.translator_en = translator_en

  def text_cleaning(self, text):
    text = re.sub('\([^)]*\)', '', text)

    return text

  def translate(self):
    df_trad = pd.DataFrame(columns=['Original', 'Translate_de_1', 'Translate_de_2'])

    df_orig = pd.read_csv(self.orig_filenamepath)

    trad_en_1 = self.translator_en([self.text_cleaning(text) for text in df_orig['Translate']])
    trad_en_2 = self.translator_en([self.text_cleaning(text) for text in df_orig['Translate_GPT_1']])

    df_trad['Original'] = df_orig['Original']
    df_trad['Translate_de_1'] = [text['translation_text'] for text in trad_en_1[:]]
    df_trad['Translate_de_2'] = [text['translation_text'] for text in trad_en_2[:]]

    df_trad.to_csv(self.trad_filenamepath, index=False, encoding='utf-8')

### Traduzione

In [9]:
# PARAMETRI GLOBALI
root_folder = 'drive/MyDrive/Traduzione_Dantesca/Data_Translate/'

# DATI
data_folder_name_originale = 'data'
filename_originale = 'generative_fiore_1.csv'

data_folder_name_traduzione = 'data_out'
filenamepath_traduzione_en = 'generative_fiore_it_de.csv'

DATA_PATH_ORIGINAL = os.path.abspath(os.path.join(root_folder, data_folder_name_originale))
orig_filenamepath = os.path.abspath(os.path.join(DATA_PATH_ORIGINAL, filename_originale))

DATA_PATH_TRANSLATE = os.path.abspath(os.path.join(root_folder, data_folder_name_traduzione))
trad_filenamepath_en = os.path.abspath(os.path.join(DATA_PATH_TRANSLATE, filenamepath_traduzione_en))

In [10]:
translate_fiore = Translate(orig_filenamepath=orig_filenamepath,
                              trad_filenamepath=trad_filenamepath_en,
                              translator_en=translator_en)

translate_fiore.translate()